# Importando as bibliotecas essenciais

Todas as bibliotecas importadas a seguir, são indispensáveis para o pré-processamento dos dados, sendo Pandas a principal delas. Nela está contida todas as formas de manipularmos dados. Também é importada a biblioteca NumPy, útil para operações com arrays.

Link do Pandas: https://pandas.pydata.org
Link do Chardet: https://pypi.org/project/chardet/
Link da NumPy: https://numpy.org

In [ ]:
import numpy as np
import pandas as pd
import chardet as chardet

# Leitura da base

A base a ser utilizada deve ser lida pelo Pandas, identificando o seu real formato, permitindo assim que a mesma possa ser lida integralmente e sem a possibilidade de erros.

Link da função read_csv: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', 30)  # Mostrar apenas 10 linhas

with open('AP.csv', 'rb') as check_raw:
    raw_data = check_raw.readline()
    encode = chardet.detect(raw_data).get('encoding')
    if encode == 'ascii':
        encode = 'utf-8'

df = pd.read_csv('AP.csv', encoding=encode, sep=None, on_bad_lines='skip', engine='python')

print("Tamanho da base: {}".format(len(df)))

# Exibição da base

Uma vez que a base foi lida com sucesso, nós devemos sempre analisar como ela está disposta e a partir disso, tomarmos as decisões para posteriores limpezas necessárias.

In [ ]:
df.head() # Por padrão, a função head() mostra apenas 5 linhas

# Breve descrição do problema

O objetivo proposto é uma análise sobre os sintomas, buscando uma correlação entre os mesmos e a possibilidade de estar infectado pelo COVID-19. Além disto, o problema também tem como objetivo secundário tentar predizer um prognóstico a respeito do grau de severidade com a qual a doença pode atingí-lo. Para isso, se faz necessário trabalhar apenas em cima de dados essenciais, tais como sintomas, comorbidades e outros dados que já são sabidamente influentes com relação a doença.

In [ ]:
df.isna().sum() # Indentificando dados nulos

In [ ]:
df.isna().sum()/len(df)*100 # Porcentagem em uma escala de 100%

# Formas de tratar dados faltantes

Existem algumas formas de tratar dados faltantes, as principais são:
- Valores numéricos: usar a média dos valores para a coluna
- Valores categóricos: usar o valor que mais aparece (moda)
- Ignorar essas linhas

# As funções que podem corresponder a estas medidas são:
- fillna(parâmetro), preenche valores faltantes com o parâmetro passado
- interpolate(), preenche o valor com base na interpolação dos valores anterior e seguinte. Link: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html

# Eliminando rows com valores nulos em colunas essenciais

As colunas definidas são de extrema importância para que os dados possam ser precisos, ainda mais pelo tipo do problema que está sendo tratado.

In [ ]:
# Colunas para analisar os dados e deletar as amostras (rows)

columns_analyze = ['evolucaoCaso', 'sintomas', 'tipoTeste', 'resultadoTeste', 'classificacaoFinal']
df.dropna(how='any', subset=columns_analyze, inplace=True)

df.head()

# Algumas correções necessárias

Algumas correções se fazem necessária pela falta de padrão em alguns preenchimentos.

In [ ]:
# No problema, os testes de antígenos são considerados com a mesma eficácia que os RT-PCR
df['tipoTeste'].replace(to_replace='TESTE RÁPIDO - ANTÍGENO', value='RT-PCR', inplace=True)
df.drop(df[df['tipoTeste'] != 'RT-PCR'].index, inplace=True) # Dropo todas os testes que sejam diferentes de RT-PCR
# Todos os casos diferentes de confirmados são deletados
df.drop(df[~df['classificacaoFinal'].str.contains(r'Confirmado')].index, inplace=True)
# Todos os resultados do teste que deram inconclusivos ou incompletos são descartados
df.drop(df[df['resultadoTeste'].str.contains(r'^[In].*')].index, inplace=True)
# Evoluções consideradas canceladas ou ignoradas são deletadas
df.drop(df[(df['evolucaoCaso'] == 'Cancelado') | (df['evolucaoCaso'] == 'Ignorado')].index, inplace=True)

df.head()

# Mais algumas correções sobre dados inconsistentes

Alguns dados não foram devidamente preenchidos, tais como quantidade de dias com sintomas e alguns outros e os mesmos são consertados via os seguintes códigos.

In [ ]:
def dates_filling(raw_df_t, *columns): # Preenche as datas faltantes
    if pd.isnull(raw_df_t.loc[columns[0]]) & pd.notnull(raw_df_t.loc[columns[1]]):
        raw_df_t[columns[0]] = raw_df_t[columns[1]]
    elif pd.notnull(raw_df_t.loc[columns[0]]) & pd.isnull(raw_df_t.loc[columns[1]]):
        raw_df_t[columns[1]] = raw_df_t[columns[0]]
    elif pd.isnull(raw_df_t.loc[columns[0]]) & pd.isnull(raw_df_t.loc[columns[1]]):
        raw_df_t[columns[0]] = pd.to_datetime('today', format='%Y-%m-%d')
        raw_df_t[columns[1]] = pd.to_datetime('today', format='%Y-%m-%d')

    return raw_df_t

# Correção, muda valores de null para empty strings.
df['condicoes'].fillna('', inplace=True)
df['outrosSintomas'].fillna('', inplace=True)

# Consertando um probleminha de datas com valores ausentes para poder aplicar funções
df = df.apply(dates_filling, args=['dataNotificacao', 'dataInicioSintomas'], axis=1)
data_notificacao = pd.to_datetime(df['dataNotificacao'], errors='coerce', format='%Y-%m-%d')
data_inicio_sintomas = pd.to_datetime(df['dataInicioSintomas'], errors='coerce', format='%Y-%m-%d')
df.insert(0, "diasSintomas", (data_notificacao - data_inicio_sintomas).dt.days, True)
df.loc[df['diasSintomas'] < 0, 'diasSintomas'] = df['diasSintomas'] * -1
df.drop(df[df['diasSintomas'] > 90].index, inplace=True)

# Colunas para deletar
columns_del = ['id', 'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao',
               'municipioNotificacaoIBGE', 'profissionalSaude', 'profissionalSeguranca', 'cbo', 'sexo', 'racaCor',
               'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'dataNotificacao', 'dataInicioSintomas',
               'estadoTeste', 'dataTeste', 'dataEncerramento', 'cnes']

# Deletando atributos irrelevantes para análise
df.drop(columns_del, axis=1, inplace=True)

df.head()

# Transformando informações de linha em colunas

Os sintomas, outros sintomas e comorbidades foram preenchidos de forma na qual eles ficam agrupados, assim, fica impossibilitado de trabalharmos com eles como features, sendo necessário transformar eles em colunas para que seja possível atuar diretamente sobre esses dados.

In [ ]:
import re # Biblioteca para aplicação de Regex
from unidecode import unidecode # Biblioteca para extrair caracteres com acentos especiais

# Sintomas e condições para virarem colunas
datas = {'other_symptoms': {'mialgia': ['mialgia', 'corpo', 'muscular'],
                            'cabeca': ['cefaleia', 'cabeca'],
                            'fadiga': ['fadiga', 'cansaco', 'fraqueza', 'indisposicao', 'adinamia', 'moleza',
                                       'astenia'],
                            'nauseas': ['nausea', 'nauseas', 'tontura', 'mal estar', 'enjoo'],
                            'coriza': ['coriza'],
                            'anosmia': ['olfato', 'anosmia', 'hiposmia'],
                            'hipogeusia': ['paladar', 'hipogeusia', 'disgeusia'],
                            'diarreia': ['diarreia'], 'febre': ['febre', 'febril'],
                            'tosse': ['tosse']
                            },
         'symptoms': ['tosse', 'febre', 'garganta', 'dispneia', 'cabeca', 'coriza', 'hipogeusia', 'anosmia',
                      'fadiga', 'nauseas', 'mialgia', 'diarreia'],
         'conditions': ['cardiacas', 'diabetes', 'respiratorias', 'renais', 'imunologica', 'obesidade',
                        'imunossupressao', 'gestante', 'puerpera']
        }

columns = datas['symptoms'] + [col for col in datas['other_symptoms'].keys() if col not in datas['symptoms']] + datas['conditions']

for i, column in enumerate(columns):
    df.insert((i + 1), column, '', True)

def standarding_features(raw_df_t, *features, **kwargs):
    column = kwargs['key']

    if not raw_df_t[column].strip():
        for i in features:
            raw_df_t[i] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos
        return raw_df_t
    else:
        for i in features:
            if re.search(i, unidecode(raw_df_t[column]).strip().lower()):  # Retira todos os caracteres especiais
                raw_df_t[i] = 1
            else:
                raw_df_t[i] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos

    return raw_df_t

# Preenche as colunas dos sintomas e condicoes com 1
for feature, column in zip([datas['symptoms'], datas['conditions']], ['sintomas', 'condicoes']):
    df = df.apply(standarding_features, axis=1, args=feature, key=column)

df.head()

# Agora operando sobre os outros sintomas

Os outros sintomas também foram colocados como um vetor, impossibilitando assim que eles sejam usados como colunas para que possam ser usados como dados para ajudar na inferência do problema.

In [ ]:
# Outros sintomas e condições para virarem colunas
def standarding_features_with_dict(raw_df_t, *column, **kwargs):
    kwargs = kwargs['key']

    if not raw_df_t[column[0]].strip():
        for key in kwargs.keys():
            if not raw_df_t[key]:
                raw_df_t[key] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos
        return raw_df_t
    else:
        for key in kwargs.keys():
            for value in kwargs[key]:
                if raw_df_t[key] == 1:
                    break
                elif re.search(value, unidecode(str(raw_df_t[column[0]])).strip().lower()):  # Retira caract. especiais
                    raw_df_t[key] = 1
                    break
                else:
                    raw_df_t[key] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos

    return raw_df_t

df = df.apply(standarding_features_with_dict, axis=1, args=['outrosSintomas'], key=datas['other_symptoms'])
df.drop(['sintomas', 'condicoes', 'outrosSintomas'], axis=1, inplace=True)
df.drop(df[(df[datas['symptoms']] == 0).all(axis=1)].index, inplace=True)

df.head()

# Última etapa do processamento para esta base

Como última etapa, iremos agregar alguns valores e substituir outros para que a nossa base fique totalmente adequada para ser trabalhada com algoritmos de aprendizagem por reforço, visando a melhor forma possível de se obter predições.

In [ ]:
# Trocar alguns valores
cols_change = ['resultadoTeste', 'evolucaoCaso']  # Índice 0 no zip
old_values = [['Negativo', 'Positivo'], [r'.*tratamento.*', r'Internado.*']]  # Índice 1 no zip
new_values = [[0, 1], ['Cura', 'Internado']]  # Índice 2 no zip
use_regex = [False, True]  # Índice 3 no zip

for values in zip(cols_change, old_values, new_values, use_regex):
    df[values[0]].replace(to_replace=values[1][0], value=values[2][0], regex=values[3], inplace=True)
    df[values[0]].replace(to_replace=values[1][1], value=values[2][1], regex=values[3], inplace=True)

for col in ['idade', 'diasSintomas']:
    df.drop(df[df[col].isin([np.nan, np.inf, -np.inf])].index, inplace=True)
    df[col] = df[col].astype(int)

df.head()

# Salvando a base pré-processada

Uma vez feito o processo de pré-processamento, agora salvamos a base para atuarmos diretamente em cima da base já consolidadamente limpa, apenas com as informações que nos sejam relevantes.

In [ ]:
df.to_csv('AP_processed.csv', sep=';', encoding='utf-8', index=False) # Salvando a base em arquivo CSV
print("Tamanho da base: {}".format(len(df)))